In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import os
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/Finnhub')

In [3]:
!pip install finnhub-python

In [4]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 10.7 MB/s eta 0:00:00


In [5]:
!pip install awscli

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 44.4 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.18.1
    Uninstalling docutils-0.18.1:
      Successfully uninstalled docutils-0.18.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [6]:
import finnhub
import json
import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import random
import time
import pickle
import boto3
from sklearn.ensemble import RandomForestClassifier
from Statistics import Statistics

In [7]:
import os
!export AWS_SHARED_CREDENTIALS_FILE=/content/gdrive/MyDrive/Colab\ Notebooks/Finnhub/awscli.ini
path = "/content/gdrive/MyDrive/Colab Notebooks/Finnhub/awscli.ini"
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path
print(os.environ['AWS_SHARED_CREDENTIALS_FILE'])

/content/gdrive/MyDrive/Colab Notebooks/Finnhub/awscli.ini


In [8]:
import os
import configparser
import boto3

# Set the environment variable
path = "/content/gdrive/MyDrive/Colab Notebooks/Finnhub/awscli.ini"

# Load credentials from the file
config = configparser.ConfigParser()
config.read(path)
credentials_section = config['default']
access_key_id = credentials_section['aws_access_key_id']
secret_access_key = credentials_section['aws_secret_access_key']

In [9]:
def extract_names_from_json(json_file_path):
    with open(json_file_path, "r") as f:
        json_data = json.load(f)

    names = []
    for entry in json_data:
        names.append(entry["Symbol"])

    return names

def tester(clf, test_data):
    dates = list(set(test_data[:,0]))
    predictions = {}
    labels = {}
    for day in dates:
        test_d = test_data[test_data[:,0]==day]
        labels[day] = test_d[:,1]
        test_d = test_d[:,2:-2]
        predictions[day] = clf.predict_proba(test_d)[:,1]
    return labels,predictions


def simulate(test_data,predictions):
    rets = pd.DataFrame([],columns=['Long','Short'])
    k = 10
    for day in sorted(predictions.keys()):
        preds = predictions[day]
        test_returns = test_data[test_data[:,0]==day][:,-2]
        top_preds = predictions[day].argsort()[-k:][::-1]
        trans_long = test_returns[top_preds]
        worst_preds = predictions[day].argsort()[:k][::-1]
        trans_short = -test_returns[worst_preds]
        rets.loc[day] = [np.mean(trans_long),np.mean(trans_short)]
    return rets

def create_stock_data(df_close,df_open,st,label,test_year):
    st_data = pd.DataFrame([])
    st_data['Date'] = list(df_close['Date'])
    st_data['Name'] = [st]*len(st_data)

    daily_change = df_close[st]/df_open[st]-1
    # print(daily_change)
    # print(len(df_close[st]))
    # print(len(df_open[st]))
    # print(sum(df_close['Date'].value_counts() == df_open['Date'].value_counts()))
    m = list(range(1,20))+list(range(20,241,20))
    for k in m:
        st_data['IntraR'+str(k)] = df_close[st].shift(1)/df_open[st].shift(k)-1
        # print(st_data['IntraR'+str(k)])
        st_data['Close' + str(k)] = df_close[st].shift(1)/df_close[st].shift(k+1)-1
        st_data['Open' + str(k)] = df_open[st]/df_close[st].shift(k) -1

    st_data['R-future'] = daily_change
    st_data['label'] = list(label[st])
    st_data['Month'] = list(df_close['Date'].str[:-3])
    # st_data.to_excel('stock_data.xlsx')
    st_data = st_data.dropna()
    # print(st_data.head())
    trade_year = st_data['Month'].str[:4]
    st_data = st_data.drop(columns=['Month'])
    st_train_data = st_data[trade_year<str(test_year)]
    # print(st_train_data)
    st_test_data = st_data[trade_year==str(test_year)]
    return np.array(st_train_data),np.array(st_test_data)

In [10]:
json_file_path = "s&p500.json"
names = extract_names_from_json(json_file_path)

In [11]:
filename = '/content/gdrive/MyDrive/Colab Notebooks/Finnhub/Open-1996.xlsx'
df_open = pd.read_excel(filename)
filename = '/content/gdrive/MyDrive/Colab Notebooks/Finnhub/Close-1996.xlsx'
df_close = pd.read_excel(filename)

In [12]:
constituents = {}

for index, row in df_open.iterrows():
  date = row['Date']
  stocks = list(row.dropna().index.drop('Date'))
  constituents[date] = stocks

In [13]:
def generate_features(df_close,df_open,cur_open,st,date):
    data = {}
    data['Date'] = date
    data['Name'] = st

    m = list(range(1,20))+list(range(20,241,20))
    for k in m:
        data['IntraR'+str(k)] = df_close.iloc[-1][st]/df_open.iloc[-k][st]-1
        # print(df_close.iloc[-1][st]/df_open.iloc[-k][st]-1)
        # print(st_data['IntraR'+str(k)])
        data['Close' + str(k)] = df_close.iloc[-1][st]/df_close.iloc[-k-1][st]-1
        # print(df_close.iloc[-1][st]/df_close.iloc[-k-1][st]-1)
        data['Open' + str(k)] = cur_open/df_close.iloc[-k][st]-1
        # print(cur_open/df_close.iloc[-k][st] -1)
    # st_data.to_excel('stock_data.xlsx')
    # st_data = st_data.dropna()
    # print(st_data.head())
    indices = [0]
    return pd.DataFrame(data, indices)

In [14]:
finnhub_client = finnhub.Client(api_key="cl056c1r01qhjei33va0cl056c1r01qhjei33vag")
s3_client = boto3.client('s3',
                         aws_access_key_id=access_key_id,
                         aws_secret_access_key=secret_access_key)

test_year = 2023
df_close_cur = df_close[(df_close['Date'].str[:4].astype(int) >= test_year -3) & (df_close['Date'].str[:4].astype(int) <= test_year)].reset_index(drop=True)
df_open_cur = df_open[(df_open['Date'].str[:4].astype(int) >= test_year - 3) & (df_open['Date'].str[:4].astype(int) <= test_year)].reset_index(drop=True)
earliest_date = ''
for key in constituents:
    if key.startswith(str(test_year-3)):
        if earliest_date == '':
          earliest_date = key
        elif key < earliest_date:
          earliest_date = key

stock_names = list(constituents[earliest_date])

start_utc = datetime.datetime(2023, 11, 6, 9, 00, 00)

end_utc = start_utc + datetime.timedelta(hours=8)

# Keep looping until the user presses a button
while True:

  # Get the current UTC time
  # end_utc = datetime.datetime.utcnow()
  stock_column_names = []
  new_open_data = []
  new_close_data = []
  # print(datetime.datetime.fromtimestamp(int(start_utc.timestamp())))
  print("START TIME")
  print(datetime.datetime.fromtimestamp(int(start_utc.timestamp())))
  print("END TIME")
  print(datetime.datetime.fromtimestamp(int(end_utc.timestamp())))
  # start_utc = end_utc - datetime.timedelta(minutes=500)
    # print(datetime.datetime.fromtimestamp(int(start_utc.timestamp())))
  date = datetime.datetime.fromtimestamp(int(end_utc.timestamp())).strftime("%Y-%m-%d")
  stock_column_names.append('Date')
  stock_column_names.extend(stock_names)

  new_open_data.append(date)
  new_close_data.append(date)

  column_names = ['Date', 'Name']
  k = list(range(1,20))+list(range(20,241,20))
  for l in k:
    column_names += ['IntraR' + str(l)]
    column_names += ['Close' + str(l)]
    column_names += ['Open' + str(l)]
  test_data = pd.DataFrame(columns = column_names)

  for stock in stock_names:
    # Get the stock candles
    candles = finnhub_client.stock_candles(stock, 60, int(start_utc.timestamp()), int(end_utc.timestamp()))

    time.sleep(.3)
    if candles['s'] == 'no_data':
      open_data = float("nan")
    else:
      open_data = candles['o'][0]
      close_data = candles['c'][-1]

    new_open_data.append(open_data)
    new_close_data.append(close_data)
    # print(generate_features(df_close_cur,df_open_cur,open_data,stock,date))
    test_data = test_data.append(generate_features(df_close_cur,df_open_cur,open_data,stock,date),ignore_index=True)

  prediction_data = np.array(test_data.dropna())[:, 2:]
  print("PREDICTION DATA SHAPE")
  print(prediction_data.shape)
  returns = np.array(new_close_data[1:]) - np.array(new_open_data[1:])
  nan_indices = np.isnan(returns)

  clean_returns = np.delete(returns, nan_indices)

  if prediction_data.size == 0:
    print('NOT A TRADING DAY')
  else:
    current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename_data = f"prediction_array_{current_time}.npy"
    filename_labels = f"labels_{current_time}.txt"
    filename_prices = f"price_array_{current_time}.npy"

    labels = list(test_data['Name'].reset_index(drop=True))

    np.save("RF_Data/" + filename_data, prediction_data)
    np.savetxt("RF_Data/" + filename_labels, labels, fmt="%s")
    np.save("RF_Data/" + filename_prices, clean_returns)

    s3_client.upload_file("RF_Data/" + filename_data, 'finnhubdata2023', 'rf-data/' + filename_data)
    s3_client.upload_file("RF_Data/" + filename_labels, 'finnhubdata2023', 'rf-data/' + filename_labels)
    s3_client.upload_file("RF_Data/" + filename_prices, 'finnhubdata2023', 'rf-data/' + filename_prices)

    new_open_dict = dict(zip(stock_column_names, new_open_data))
    new_close_dict = dict(zip(stock_column_names, new_close_data))

    df_close_cur = df_close_cur.append(new_close_dict, ignore_index=True)
    df_open_cur = df_open_cur.append(new_open_dict, ignore_index=True)

  start_utc = start_utc + datetime.timedelta(days = 1)
  end_utc = start_utc + datetime.timedelta(hours=8)
  time.sleep(100)

START TIME
2023-11-06 09:00:00
END TIME
2023-11-06 17:00:00
PREDICTION DATA SHAPE
(476, 93)
START TIME
2023-11-07 09:00:00
END TIME
2023-11-07 17:00:00
PREDICTION DATA SHAPE
(476, 93)
START TIME
2023-11-08 09:00:00
END TIME
2023-11-08 17:00:00
PREDICTION DATA SHAPE
(476, 93)
START TIME
2023-11-09 09:00:00
END TIME
2023-11-09 17:00:00
PREDICTION DATA SHAPE
(476, 93)
START TIME
2023-11-10 09:00:00
END TIME
2023-11-10 17:00:00
PREDICTION DATA SHAPE
(476, 93)
START TIME
2023-11-11 09:00:00
END TIME
2023-11-11 17:00:00
PREDICTION DATA SHAPE
(0, 93)
NOT A TRADING DAY
START TIME
2023-11-12 09:00:00
END TIME
2023-11-12 17:00:00
PREDICTION DATA SHAPE
(0, 93)
NOT A TRADING DAY
START TIME
2023-11-13 09:00:00
END TIME
2023-11-13 17:00:00
PREDICTION DATA SHAPE
(476, 93)
START TIME
2023-11-14 09:00:00
END TIME
2023-11-14 17:00:00
PREDICTION DATA SHAPE
(476, 93)
START TIME
2023-11-15 09:00:00
END TIME
2023-11-15 17:00:00
PREDICTION DATA SHAPE
(476, 93)
START TIME
2023-11-16 09:00:00
END TIME
2023-11-

KeyboardInterrupt: ignored

In [ ]:
with open('random_forest_model.pkl', 'rb') as f:
  data = pickle.load(f)

data.n_features_in_